In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def get_economic_cycle_indicators():
    """
    Coleta indicadores econômicos para determinar o ciclo vigente
    """
    # Período de análise (últimos 2 anos)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=730)
    
    indicators = {}
    
    try:
        print("Coletando dados dos Treasuries...")
        # 1. Curva de Juros (10Y-2Y Treasury Spread)
        treasury_10y = yf.download("^TNX", start=start_date, end=end_date, progress=False)
        treasury_2y = yf.download("^IRX", start=start_date, end=end_date, progress=False)
        
        if not treasury_10y.empty and not treasury_2y.empty and 'Close' in treasury_10y.columns and 'Close' in treasury_2y.columns:
            # Pegar apenas os valores válidos (não-NaN)
            t10_clean = treasury_10y['Close'].dropna()
            t2_clean = treasury_2y['Close'].dropna()
            
            if len(t10_clean) > 0 and len(t2_clean) > 0:
                # Usar o último valor disponível
                yield_10y = float(t10_clean.iloc[-1])
                yield_2y = float(t2_clean.iloc[-1])
                
                indicators['yield_curve_current'] = yield_10y - yield_2y
                indicators['yield_curve_trend'] = 'Invertida' if indicators['yield_curve_current'] < 0 else 'Normal'
                print(f"✅ Treasuries coletados: 10Y={yield_10y:.2f}%, 2Y={yield_2y:.2f}%")
            else:
                print("⚠️ Dados de Treasury sem valores válidos")
        else:
            print("⚠️ Erro ao baixar dados de Treasury")
        
        # 2. S&P 500 Performance e Volatilidade
        sp500 = yf.download("^GSPC", start=start_date, end=end_date, progress=False)
        if not sp500.empty:
            sp500_returns = sp500['Close'].pct_change().dropna()
            
            indicators['sp500_ytd_return'] = float(((sp500['Close'].iloc[-1] / sp500['Close'].iloc[0]) - 1) * 100)
            indicators['sp500_volatility'] = float(sp500_returns.std() * np.sqrt(252) * 100)  # Anualizada
        
        # 3. VIX (Fear Index)
        vix = yf.download("^VIX", start=start_date, end=end_date, progress=False)['Close']
        if not vix.empty:
            indicators['vix_current'] = float(vix.iloc[-1])
            indicators['vix_avg_3m'] = float(vix.tail(63).mean()) if len(vix) >= 63 else np.nan
        
        # 4. Dollar Index (DXY)
        dxy = yf.download("DX-Y.NYB", start=start_date, end=end_date, progress=False)['Close']
        if not dxy.empty:
            indicators['dxy_current'] = float(dxy.iloc[-1])
            indicators['dxy_change_3m'] = float(((dxy.iloc[-1] / dxy.iloc[-63]) - 1) * 100) if len(dxy) >= 63 else np.nan
        
        # 5. Commodities (Oil - WTI)
        oil = yf.download("CL=F", start=start_date, end=end_date, progress=False)['Close']
        if not oil.empty:
            indicators['oil_current'] = float(oil.iloc[-1])
            indicators['oil_change_3m'] = float(((oil.iloc[-1] / oil.iloc[-63]) - 1) * 100) if len(oil) >= 63 else np.nan
        
        # 6. High Yield vs Treasury Spread
        hy_bond = yf.download("HYG", start=start_date, end=end_date, progress=False)['Close']
        treasury_etf = yf.download("IEF", start=start_date, end=end_date, progress=False)['Close']
        
        if not hy_bond.empty and not treasury_etf.empty:
            hy_returns = hy_bond.pct_change().dropna()
            treasury_returns = treasury_etf.pct_change().dropna()
            
            # Correlação como proxy para spread de crédito
            if len(hy_returns) > 30 and len(treasury_returns) > 30:
                common_dates = hy_returns.index.intersection(treasury_returns.index)
                if len(common_dates) > 30:
                    correlation = hy_returns[common_dates].corr(treasury_returns[common_dates])
                    indicators['credit_risk_signal'] = 'Alto' if correlation < -0.3 else 'Baixo'
        
    except Exception as e:
        print(f"Erro ao coletar dados: {e}")
    
    return indicators

def determine_economic_cycle(indicators):
    """
    Determina o ciclo econômico baseado nos indicadores
    """
    score = 0
    signals = []
    
    # Análise da Curva de Juros
    if 'yield_curve_current' in indicators:
        if indicators['yield_curve_current'] < -0.5:
            score -= 2
            signals.append("⚠️ Curva de juros invertida (recessão)")
        elif indicators['yield_curve_current'] < 0:
            score -= 1
            signals.append("⚠️ Curva de juros levemente invertida")
        elif indicators['yield_curve_current'] > 2:
            score += 1
            signals.append("✅ Curva de juros normal e íngreme")
    
    # Análise do VIX
    if 'vix_current' in indicators:
        if indicators['vix_current'] > 30:
            score -= 2
            signals.append("⚠️ VIX alto - medo no mercado")
        elif indicators['vix_current'] > 20:
            score -= 1
            signals.append("⚠️ VIX moderado - alguma tensão")
        else:
            score += 1
            signals.append("✅ VIX baixo - mercado tranquilo")
    
    # Análise do S&P 500
    if 'sp500_ytd_return' in indicators:
        if indicators['sp500_ytd_return'] < -15:
            score -= 2
            signals.append("⚠️ S&P 500 em bear market")
        elif indicators['sp500_ytd_return'] < -5:
            score -= 1
            signals.append("⚠️ S&P 500 com performance negativa")
        elif indicators['sp500_ytd_return'] > 15:
            score += 1
            signals.append("✅ S&P 500 com forte performance")
    
    # Análise de Volatilidade
    if 'sp500_volatility' in indicators:
        if indicators['sp500_volatility'] > 25:
            score -= 1
            signals.append("⚠️ Alta volatilidade no mercado")
        elif indicators['sp500_volatility'] < 15:
            score += 1
            signals.append("✅ Baixa volatilidade")
    
    # Determinação do ciclo
    if score >= 2:
        cycle = "🟢 EXPANSÃO"
        description = "Economia em crescimento, mercados otimistas"
        recommendations = [
            "Foco em growth stocks e tecnologia",
            "Reduzir posição em bonds de longo prazo",
            "Considerar REITs e commodities",
            "Aumentar exposição a small caps"
        ]
    elif score >= 0:
        cycle = "🟡 EXPANSÃO TARDIA"
        description = "Crescimento desacelerando, inflação possível"
        recommendations = [
            "Diversificar entre growth e value",
            "Considerar TIPS (inflation-protected securities)",
            "Manter posição defensiva em utilities",
            "Reduzir duration em bonds"
        ]
    elif score >= -2:
        cycle = "🟠 DESACELERAÇÃO"
        description = "Economia desacelerando, mercados cautelosos"
        recommendations = [
            "Migrar para value stocks e dividendos",
            "Aumentar posição em bonds de alta qualidade",
            "Considerar setores defensivos (utilities, consumer staples)",
            "Reduzir exposição a growth e tecnologia"
        ]
    else:
        cycle = "🔴 RECESSÃO"
        description = "Contração econômica, mercados em stress"
        recommendations = [
            "Posição defensiva: cash e treasuries",
            "Foco em dividend aristocrats",
            "Evitar high yield e junk bonds",
            "Considerar inverse ETFs para hedge"
        ]
    
    return {
        'cycle': cycle,
        'description': description,
        'score': score,
        'signals': signals,
        'recommendations': recommendations
    }

def display_cycle_analysis():
    """
    Executa a análise completa e exibe os resultados
    """
    print("🔍 ANALISANDO CICLO ECONÔMICO DO MERCADO AMERICANO...")
    print("=" * 60)
    
    # Coleta indicadores
    indicators = get_economic_cycle_indicators()
    
    # Exibe indicadores principais
    print("\n📊 INDICADORES ECONÔMICOS ATUAIS:")
    print("-" * 40)
    
    if 'yield_curve_current' in indicators and not np.isnan(indicators['yield_curve_current']):
        print(f"Curva de Juros (10Y-2Y): {indicators['yield_curve_current']:.2f}% ({indicators['yield_curve_trend']})")
    
    if 'vix_current' in indicators and not np.isnan(indicators['vix_current']):
        print(f"VIX (Índice do Medo): {indicators['vix_current']:.1f}")
    
    if 'sp500_ytd_return' in indicators and not np.isnan(indicators['sp500_ytd_return']):
        print(f"S&P 500 YTD: {indicators['sp500_ytd_return']:.1f}%")
    
    if 'sp500_volatility' in indicators and not np.isnan(indicators['sp500_volatility']):
        print(f"Volatilidade S&P 500: {indicators['sp500_volatility']:.1f}%")
    
    if 'dxy_current' in indicators and not np.isnan(indicators['dxy_current']):
        print(f"Índice do Dólar (DXY): {indicators['dxy_current']:.1f}")
    
    if 'oil_current' in indicators and not np.isnan(indicators['oil_current']):
        print(f"Petróleo WTI: ${indicators['oil_current']:.1f}")
    
    # Determina ciclo
    cycle_analysis = determine_economic_cycle(indicators)
    
    print(f"\n🎯 CICLO ECONÔMICO IDENTIFICADO:")
    print("-" * 40)
    print(f"Status: {cycle_analysis['cycle']}")
    print(f"Descrição: {cycle_analysis['description']}")
    print(f"Score de Confiança: {cycle_analysis['score']}")
    
    print(f"\n🔍 SINAIS IDENTIFICADOS:")
    print("-" * 40)
    for signal in cycle_analysis['signals']:
        print(f"  {signal}")
    
    print(f"\n💡 RECOMENDAÇÕES PARA CARTEIRA:")
    print("-" * 40)
    for i, rec in enumerate(cycle_analysis['recommendations'], 1):
        print(f"  {i}. {rec}")
    
    print("\n" + "=" * 60)
    print("✅ Análise concluída! Use essas informações para ajustar sua estratégia de alocação.")
    
    return cycle_analysis, indicators

# Executar análise
if __name__ == "__main__":
    cycle_result, raw_indicators = display_cycle_analysis()

🔍 ANALISANDO CICLO ECONÔMICO DO MERCADO AMERICANO...
Coletando dados dos Treasuries...
✅ Treasuries coletados: 10Y=4.20%, 2Y=4.16%
Erro ao coletar dados: "None of [DatetimeIndex(['2023-08-08', '2023-08-09', '2023-08-10', '2023-08-11',\n               '2023-08-14', '2023-08-15', '2023-08-16', '2023-08-17',\n               '2023-08-18', '2023-08-21',\n               ...\n               '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25',\n               '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31',\n               '2025-08-01', '2025-08-04'],\n              dtype='datetime64[ns]', name='Date', length=499, freq=None)] are in the [columns]"

📊 INDICADORES ECONÔMICOS ATUAIS:
----------------------------------------
Curva de Juros (10Y-2Y): 0.04% (Normal)
VIX (Índice do Medo): 17.9
S&P 500 YTD: 39.8%
Volatilidade S&P 500: 16.1%
Índice do Dólar (DXY): 98.8
Petróleo WTI: $66.3

🎯 CICLO ECONÔMICO IDENTIFICADO:
----------------------------------------
Status: 🟢 EXPANSÃO
Descrição: E

In [24]:
import os

# Substitua abaixo pela sua chave real:
os.environ['FRED_API_KEY'] = "7602138fb8830fa148645d69bc144ba5"

# Verifique se está ok:
print("FRED_API_KEY =", os.getenv("FRED_API_KEY"))


FRED_API_KEY = 7602138fb8830fa148645d69bc144ba5


In [ ]:
import os
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import warnings

warnings.filterwarnings('ignore')

# —————————————— CONFIGURAÇÃO ——————————————
FRED_API_KEY = os.getenv("FRED_API_KEY")
if not FRED_API_KEY:
    raise ValueError("Defina a variável de ambiente FRED_API_KEY com sua chave do FRED")

END   = datetime.now()
START = END - timedelta(days=730)

# séries do FRED
FRED_SERIES = {
    'gdp_real':  'GDPC1',     # PIB real trimestral
    'gdp_pot':   'GDPPOT',    # PIB potencial trimestral
    'cap_util':  'TCU',       # Utilização de capacidade mensal
    'cpi':       'PCEPI',  # indice PCE total
    'rate_10y':  'DGS10',     # Yield 10 anos diário
    'rate_2y':   'DGS2'       # Yield 2 anos diário
}

def fetch_fred(series_id: str, start: datetime, end: datetime) -> pd.Series:
    """Busca observações de uma série do FRED em JSON e retorna pd.Series."""
    url = (
        "https://api.stlouisfed.org/fred/series/observations"
        f"?series_id={series_id}"
        f"&api_key={FRED_API_KEY}"
        f"&file_type=json"
        f"&observation_start={start.strftime('%Y-%m-%d')}"
        f"&observation_end={end.strftime('%Y-%m-%d')}"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    observations = resp.json()['observations']
    dates = pd.to_datetime([o['date'] for o in observations])
    values = [float(o['value']) if o['value'] not in ('.','') else np.nan for o in observations]
    return pd.Series(values, index=dates)

def fetch_indicators():
    data = {}
    # 1) PIB real & potencial → hiato (último trimestre disponível)
    gdp     = fetch_fred(FRED_SERIES['gdp_real'], START, END).dropna()
    gdp_pot = fetch_fred(FRED_SERIES['gdp_pot'],  START, END).dropna()
    last_date = gdp.index.intersection(gdp_pot.index)[-1]
    gap = (gdp.loc[last_date] / gdp_pot.loc[last_date] - 1) * 100
    data['gdp_gap'] = float(gap)

    # 2) Utilização de capacidade (último valor e média móvel 12m)
    cap = fetch_fred(FRED_SERIES['cap_util'], START, END).dropna()
    data['cap_util_current'] = float(cap.iloc[-1])
    data['cap_util_trend']   = float(cap.rolling(12).mean().iloc[-1])

    # 3) Inflação YoY (CPI)
    cpi = fetch_fred(FRED_SERIES['cpi'], START, END).dropna()
    yoy = (cpi.iloc[-1] / cpi.shift(12).iloc[-1] - 1) * 100
    data['inflation_yoy'] = float(yoy)

    # 4) Curva de juros 10Y–2Y
    r10 = fetch_fred(FRED_SERIES['rate_10y'], START, END).dropna()
    r2  = fetch_fred(FRED_SERIES['rate_2y'],  START, END).dropna()
    date_common = r10.index.intersection(r2.index)[-1]
    data['yield_curve'] = float(r10.loc[date_common] - r2.loc[date_common])

    # 5) DXY via yfinance
    dxy = yf.download("DX-Y.NYB", start=START, end=END, progress=False)['Close'].dropna()
    data['dxy_current']   = float(dxy.iloc[-1])
    if len(dxy) >= 63:
        data['dxy_change_3m'] = float((dxy.iloc[-1] / dxy.iloc[-63] - 1) * 100)

    return data

def determine_phase(ind):
    INFLATION_THRESHOLD = 3.0
    gap, inf = ind['gdp_gap'], ind['inflation_yoy']
    if gap <  0 and inf <  2:
        phase = 'Reflação'
    elif gap >= 0 and inf <  INFLATION_THRESHOLD:
        phase = 'Recuperação'
    elif gap >= 0 and inf >= INFLATION_THRESHOLD:
        phase = 'Superaquecimento'

    else:
        phase = 'Estagflação'

    mapping = {
        'Reflação': {
            'asset':   'Treasuries (bonds)',
            'sectors': ['Financials', 'Consumer Discretionary e Pharmaceuticals','Consumer Discretionary']
        },
        'Recuperação': {
            'asset':   'Equities (Stocks)',
            'sectors': ['Telecoms', 'Information Technology e Basic Materials', 'Consumer Discretionary']
        },
        'Superaquecimento': {
            'asset':   'Commodities',
            'sectors': ['Industrials','Information Technology e Basic Materials', 'Oil & Gas']
        },
        'Estagflação': {
            'asset':   'Cash (dinheiro)',
            'sectors': ['Utilities', 'Consumer Discretionary e Pharmaceuticals', 'Oil & Gas']
        }
    }

    info = mapping[phase]
    return {
        'phase':       phase,
        'asset_class': info['asset'],
        'sector_recs': info['sectors']
    }


def display_investment_clock():
    print("🔍 Relógio de Investimentos (Merrill Lynch)")
    print("="*60)
    ind = fetch_indicators()
    print(f"• Hiato do PIB (últ. trimestre): {ind['gdp_gap']:.2f}%")
    print(f"• Inflação YoY (CPI):            {ind['inflation_yoy']:.2f}%")
    print(f"• Curva Juros (10Y–2Y):          {ind['yield_curve']:.2f} pp")
    print(f"• Utiliz. capacidade:            {ind['cap_util_current']:.1f}% (Trend: {ind['cap_util_trend']:.1f}%)")
    print(f"• DXY Atual:                     {ind['dxy_current']:.2f}")
    phase = determine_phase(ind)
    print("\n🎯 Fase atual:")
    print(f"   • {phase['phase']}")
    print(f"   • Classe de ativo-chave: {phase['asset_class']}")
    print(f"   • Setores: {', '.join(phase['sector_recs'])}")
    print("="*60)
    return ind, phase

if __name__ == "__main__":
    display_investment_clock()


🔍 Relógio de Investimentos (Merrill Lynch)
• Hiato do PIB (últ. trimestre): 1.32%
• Inflação YoY (CPI):            2.58%
• Curva Juros (10Y–2Y):          0.54 pp
• Utiliz. capacidade:            77.6% (Trend: 77.5%)
• DXY Atual:                     98.99

🎯 Fase atual:
   • Recuperação
   • Classe de ativo-chave: Equities
   • Setores: Telecoms, Information Technology e Basic Materials, Consumer Discretionary


In [3]:
# weekly_portfolio_report.py - Desenvolvimento em blocos com .env

# === Bloco 1: Importação das bibliotecas necessárias ===
import os
import glob
import requests
import pandas as pd
import schedule
import time
from datetime import datetime
from dotenv import load_dotenv

# Carrega variáveis do .env
load_dotenv()

# Chave da API FMP obtida de variáveis de ambiente
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError('A variável de ambiente FMP_API_KEY não está definida')

# === Bloco 2: Configurações e carregamento de scores ===
SCORES_FOLDER = 'ativos'      # Pasta com CSVs de scores
SCORE_THRESHOLD = 50.0        # Limite mínimo de score


def load_and_filter_scores(folder: str, threshold: float = SCORE_THRESHOLD) -> pd.DataFrame:
    """
    Lê CSVs em folder, extrai Ticker e Score, atribui Setor (nome do arquivo),
    filtra Score > threshold e retorna DataFrame ordenado por Score.
    Colunas: Ticker, Score, Setor.
    """
    csv_files = glob.glob(os.path.join(folder, '*.csv'))
    if not csv_files:
        raise FileNotFoundError(f"Nenhum CSV encontrado em {folder}")

    registros = []
    for fn in csv_files:
        setor = os.path.splitext(os.path.basename(fn))[0]
        df = pd.read_csv(fn, header=None, skiprows=1, usecols=[0,1],
                         names=['Ticker','Score'], decimal=',')
        df['Ticker'] = df['Ticker'].astype(str).str.strip()
        df['Score'] = pd.to_numeric(df['Score'], errors='coerce')
        df['Setor'] = setor
        registros.extend(
            {'Ticker': r.Ticker, 'Score': r.Score, 'Setor': r.Setor}
            for r in df.itertuples() if pd.notnull(r.Score)
        )

    df_all = pd.DataFrame(registros)
    df_filtered = df_all[df_all['Score'] > threshold]
    return df_filtered.sort_values('Score', ascending=False).reset_index(drop=True)

# === Bloco 3: Classificação de tipo de ativo ===
FMP_PROFILE_URL = "https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api_key}"
CACHE = {}

def fetch_asset_profile(ticker: str) -> dict:
    t = ticker.upper()
    if t in CACHE:
        return CACHE[t]
    try:
        resp = requests.get(FMP_PROFILE_URL.format(ticker=t, api_key=FMP_API_KEY))
        data = resp.json()
        prof = data[0] if isinstance(data, list) and data else {}
    except:
        prof = {}
    CACHE[t] = prof
    return prof


def classify_type(profile: dict) -> str:
    mkt = profile.get('mktCap', 0)
    ind = (profile.get('industry') or '').lower()
    name = (profile.get('companyName') or '').lower()
    if 'etf' in ind or 'etf' in name:
        return 'ETF'
    if 'real estate' in ind or 'reit' in ind:
        return 'REIT'
    if isinstance(mkt, (int,float)):
        if mkt > 50e9:
            return 'Blue Chip'
        if mkt > 2e9:
            return 'Mid Cap'
        return 'Small Cap'
    return 'Unknown'

# === Bloco 4: Geração de ranking de Ações e REITs ===
def generate_report():
    """
    Gera top 14 Ações e top 14 REITs baseados em Score.
    """
    df = load_and_filter_scores(SCORES_FOLDER)
    df['Tipo'] = df['Ticker'].apply(lambda t: classify_type(fetch_asset_profile(t)))

    # Top 7 Ações (exclui REITs e ETFs)
    stocks = df[~df['Tipo'].isin(['ETF','REIT'])].head(14).copy()
    stocks['Rank'] = range(1, len(stocks)+1)

    # Top 7 REITs
    reits = df[df['Tipo']=='REIT'].head(14).copy()
    reits['Rank'] = range(1, len(reits)+1)

    print('\n=== Top 14 Ações ===')
    print(stocks[['Rank','Ticker','Score','Setor','Tipo']].to_string(index=False))

    print('\n=== Top 14 REITs ===')
    if not reits.empty:
        print(reits[['Rank','Ticker','Score','Setor','Tipo']].to_string(index=False))
    else:
        print('Nenhum REIT com Score acima do threshold')

    return stocks, reits

# === Bloco 5: Agendamento semanal ===
def schedule_weekly_report(day='monday', time_str='09:00'):
    getattr(schedule.every(), day).at(time_str).do(generate_report)
    print(f'Relatório agendado: toda {day} às {time_str}')

if __name__=='__main__':
    print('Iniciando relatório...')
    top_stocks, top_reits = generate_report()
    # schedule_weekly_report()
    # while True: schedule.run_pending(); time.sleep(60)


python-dotenv could not parse statement starting at line 8


Iniciando relatório...

=== Top 14 Ações ===
 Rank Ticker  Score                  Setor      Tipo
    1    CRM  73.88 Information Technology Blue Chip
    2   RACE  70.87 Consumer Discretionary Blue Chip
    3   CARR  69.16            Industrials Blue Chip
    4    YUM  67.83 Consumer Discretionary   Mid Cap
    5     WM  67.41            Industrials Blue Chip
    6    AON  66.01             Financials Blue Chip
    7      V  65.73 Information Technology Blue Chip
    8    TJX  65.73 Consumer Discretionary Blue Chip
    9   FAST  65.58            Industrials Blue Chip
   10    WAB  64.76            Industrials   Mid Cap
   11    MCD  64.00 Consumer Discretionary Blue Chip
   12   ORLY  63.65 Consumer Discretionary Blue Chip
   13   MNST  63.64 Consumer Non-Cyclicals Blue Chip
   14   TMUS  62.88 Information Technology Blue Chip

=== Top 14 REITs ===
 Rank Ticker  Score       Setor Tipo
    1   EQIX  67.49 Real Estate REIT
    2      O  64.39 Real Estate REIT
    3   BEKE  61.99 Real Es

In [4]:
# weekly_portfolio_report.py - Desenvolvimento em blocos com .env e filtro por setores do ciclo

# === Bloco 1: Importação das bibliotecas necessárias ===
import os
import glob
import requests
import pandas as pd
import numpy as np
import schedule
import time
from datetime import datetime, timedelta
import warnings
import yfinance as yf
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Carrega variáveis do .env
load_dotenv()

# Chaves de API obtidas de variáveis de ambiente
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError('A variável de ambiente FMP_API_KEY não está definida')
FRED_API_KEY = os.getenv('FRED_API_KEY')
if not FRED_API_KEY:
    raise ValueError('A variável de ambiente FRED_API_KEY não está definida')

# Datas para indicadores
END   = datetime.now()
START = END - timedelta(days=730)

# Séries do FRED
FRED_SERIES = {
    'gdp_real':  'GDPC1',
    'gdp_pot':   'GDPPOT',
    'cap_util':  'TCU',
    'cpi':       'PCEPI',
    'rate_10y':  'DGS10',
    'rate_2y':   'DGS2'
}

# === Bloco 1.5: Funções do ciclo econômico ===

def fetch_fred(series_id: str, start: datetime, end: datetime) -> pd.Series:
    url = (
        'https://api.stlouisfed.org/fred/series/observations'
        f'?series_id={series_id}'
        f'&api_key={FRED_API_KEY}'
        '&file_type=json'
        f'&observation_start={start.strftime('%Y-%m-%d')}'
        f'&observation_end={end.strftime('%Y-%m-%d')}'
    )
    resp = requests.get(url)
    resp.raise_for_status()
    obs = resp.json().get('observations', [])
    dates = pd.to_datetime([o['date'] for o in obs])
    values = [float(o['value']) if o['value'] not in ('.','') else np.nan for o in obs]
    return pd.Series(values, index=dates)


def fetch_indicators():
    data = {}
    # PIB real vs potencial
    gdp     = fetch_fred(FRED_SERIES['gdp_real'], START, END).dropna()
    gdp_pot = fetch_fred(FRED_SERIES['gdp_pot'],  START, END).dropna()
    last = gdp.index.intersection(gdp_pot.index)[-1]
    data['gdp_gap'] = (gdp.loc[last]/gdp_pot.loc[last] - 1) * 100
    # Utilização de capacidade
    cap = fetch_fred(FRED_SERIES['cap_util'], START, END).dropna()
    data['cap_util_current'] = cap.iloc[-1]
    data['cap_util_trend']   = cap.rolling(12).mean().iloc[-1]
    # Inflação YoY
    cpi = fetch_fred(FRED_SERIES['cpi'], START, END).dropna()
    data['inflation_yoy'] = (cpi.iloc[-1]/cpi.shift(12).iloc[-1] - 1) * 100
    # Curva de juros
    r10 = fetch_fred(FRED_SERIES['rate_10y'], START, END).dropna()
    r2  = fetch_fred(FRED_SERIES['rate_2y'],  START, END).dropna()
    common = r10.index.intersection(r2.index)[-1]
    data['yield_curve'] = r10.loc[common] - r2.loc[common]
    # DXY
    dxy = yf.download('DX-Y.NYB', start=START, end=END, progress=False)['Close'].dropna()
    data['dxy_current'] = dxy.iloc[-1]
    return {k: float(v) for k,v in data.items()}


def determine_phase(ind):
    inf = ind['inflation_yoy']
    gap = ind['gdp_gap']
    if gap < 0 and inf < 2:
        phase = 'Reflação'
    elif gap >= 0 and inf < 3:
        phase = 'Recuperação'
    elif gap >= 0 and inf >= 3:
        phase = 'Superaquecimento'
    else:
        phase = 'Estagflação'
    mapping = {
        'Reflação': {'asset':'Treasuries','sectors':['Financials','Consumer Discretionary','Pharmaceuticals']},
        'Recuperação':{'asset':'Equities','sectors':['Telecoms','Information Technology','Basic Materials','Consumer Discretionary']},
        'Superaquecimento':{'asset':'Commodities','sectors':['Industrials','Information Technology','Basic Materials','Oil & Gas']},
        'Estagflação':{'asset':'Cash','sectors':['Utilities','Consumer Discretionary','Pharmaceuticals','Oil & Gas']}
    }
    info = mapping[phase]
    return {'phase':phase,'asset_class':info['asset'],'sector_recs':info['sectors']}


def display_investment_clock():
    print('🔍 Relógio de Investimentos (Merrill Lynch)')
    print('='*60)
    ind = fetch_indicators()
    print(f"• Hiato do PIB: {ind['gdp_gap']:.2f}% | Inflação YoY: {ind['inflation_yoy']:.2f}% | Curva Juros: {ind['yield_curve']:.2f} pp")
    print(f"• Utilização capacidade: {ind['cap_util_current']:.1f}% (Trend {ind['cap_util_trend']:.1f}%) | DXY: {ind['dxy_current']:.2f}")
    phase = determine_phase(ind)
    print('\n🎯 Fase atual:')
    print(f"   • {phase['phase']} ({phase['asset_class']})")
    print('   • Setores:', ', '.join(phase['sector_recs']))
    print('='*60)
    return ind, phase

# === Bloco 2: Configurações e carregamento de scores ===
SCORES_FOLDER = 'ativos'
SCORE_THRESHOLD = 50.0

def load_and_filter_scores(folder: str, threshold: float = SCORE_THRESHOLD) -> pd.DataFrame:
    files = glob.glob(os.path.join(folder,'*.csv'))
    registros = []
    for fn in files:
        setor = os.path.splitext(os.path.basename(fn))[0]
        df = pd.read_csv(fn, header=None, skiprows=1, usecols=[0,1], names=['Ticker','Score'], decimal=',')
        df['Score'] = pd.to_numeric(df['Score'], errors='coerce')
        df['Setor'] = setor
        registros += [ {'Ticker':r.Ticker,'Score':r.Score,'Setor':setor} for r in df.itertuples() if pd.notnull(r.Score) ]
    df = pd.DataFrame(registros)
    return df[df['Score']>threshold].sort_values('Score',ascending=False).reset_index(drop=True)[['Ticker','Score','Setor']]

# === Bloco 3: Classificação tipo de ativo ===
FMP_PROFILE_URL = 'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api_key}'
CACHE = {}

def fetch_asset_profile(ticker):
    t = ticker.upper()
    if t in CACHE: return CACHE[t]
    try:
        resp = requests.get(FMP_PROFILE_URL.format(ticker=t,api_key=FMP_API_KEY)); data = resp.json()
        prof = data[0] if isinstance(data,list) and data else {}
    except: prof={}
    CACHE[t]=prof; return prof

def classify_type(profile):
    mkt=profile.get('mktCap',0); ind=(profile.get('industry')or'').lower(); name=(profile.get('companyName')or'').lower()
    if 'etf' in ind or 'etf' in name: return 'ETF'
    if 'real estate' in ind or 'reit' in ind: return 'REIT'
    if isinstance(mkt,(int,float)):
        if mkt>50e9: return 'Blue Chip'
        if mkt>2e9: return 'Mid Cap'
        return 'Small Cap'
    return 'Unknown'

# === Bloco 4: Geração de ranking filtrado por setores do ciclo econômico ===
def generate_report():
    _,phase=display_investment_clock(); sectors=phase['sector_recs']
    print(f"Setores do ciclo: {', '.join(sectors)}")
    df=load_and_filter_scores(SCORES_FOLDER)
    df=df[df['Setor'].isin(sectors)].reset_index(drop=True)
    df['Tipo']=df['Ticker'].apply(lambda t: classify_type(fetch_asset_profile(t)))
    stocks=df[~df['Tipo'].isin(['ETF','REIT'])].head(14).copy(); stocks['Rank']=range(1,len(stocks)+1)
    reits=df[df['Tipo']=='REIT'].head(14).copy(); reits['Rank']=range(1,len(reits)+1)
    print('\n=== Top 14 Ações ===', stocks[['Rank','Ticker','Score','Setor','Tipo']].to_string(index=False) if not stocks.empty else 'Nenhuma ação')
    print('\n=== Top 14 REITs ===', reits[['Rank','Ticker','Score','Setor','Tipo']].to_string(index=False) if not reits.empty else 'Nenhum REIT')
    return stocks,reits

# === Bloco 5: Agendamento semanal ===
def schedule_weekly_report(day='monday',time_str='09:00'):
    getattr(schedule.every(),day).at(time_str).do(generate_report)
    print(f'Relatório agendado: toda {day} às {time_str}')

if __name__=='__main__':
    print('Iniciando relatório...')
    generate_report()


python-dotenv could not parse statement starting at line 8


Iniciando relatório...
🔍 Relógio de Investimentos (Merrill Lynch)
• Hiato do PIB: 1.32% | Inflação YoY: 2.58% | Curva Juros: 0.54 pp
• Utilização capacidade: 77.6% (Trend 77.5%) | DXY: 98.96

🎯 Fase atual:
   • Recuperação (Equities)
   • Setores: Telecoms, Information Technology, Basic Materials, Consumer Discretionary
Setores do ciclo: Telecoms, Information Technology, Basic Materials, Consumer Discretionary

=== Top 14 Ações ===  Rank Ticker  Score                  Setor      Tipo
    1    CRM  73.88 Information Technology Blue Chip
    2   RACE  70.87 Consumer Discretionary Blue Chip
    3    YUM  67.83 Consumer Discretionary   Mid Cap
    4      V  65.73 Information Technology Blue Chip
    5    TJX  65.73 Consumer Discretionary Blue Chip
    6    MCD  64.00 Consumer Discretionary Blue Chip
    7   ORLY  63.65 Consumer Discretionary Blue Chip
    8   TMUS  62.88 Information Technology Blue Chip
    9    ADI  62.53 Information Technology Blue Chip
   10     IP  61.39        Basic M

In [5]:
# weekly_portfolio_report.py - Desenvolvimento em blocos com .env e filtro por ciclos econômicos

# === Bloco 1: Importação das bibliotecas necessárias ===
import os
import glob
import requests
import pandas as pd
import numpy as np
import schedule
import time
from datetime import datetime, timedelta
import warnings
import yfinance as yf
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# === Bloco 1.1: Carrega variáveis de ambiente ===
load_dotenv()
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError('Defina FMP_API_KEY no .env')
FRED_API_KEY = os.getenv('FRED_API_KEY')
if not FRED_API_KEY:
    raise ValueError('Defina FRED_API_KEY no .env')

# === Bloco 1.2: Configurações ciclo econômico ===
END = datetime.now()
START = END - timedelta(days=730)

FRED_SERIES = {
    'gdp_real': 'GDPC1', 'gdp_pot': 'GDPPOT', 'cap_util': 'TCU',
    'cpi': 'PCEPI', 'rate_10y': 'DGS10', 'rate_2y': 'DGS2'
}
CYCLE_MAPPING = {
    'Reflação': {
        'asset': 'Treasuries',
        'sectors': ['Financials', 'Consumer Discretionary', 'Pharmaceuticals']
    },
    'Recuperação': {
        'asset': 'Equities',
        'sectors': ['Telecoms', 'Information Technology', 'Basic Materials', 'Consumer Discretionary']
    },
    'Superaquecimento': {
        'asset': 'Commodities',
        'sectors': ['Industrials', 'Information Technology', 'Basic Materials', 'Oil & Gas']
    },
    'Estagflação': {
        'asset': 'Cash',
        'sectors': ['Utilities', 'Consumer Discretionary', 'Pharmaceuticals', 'Oil & Gas']
    }
}
CYCLE_ORDER = ['Reflação', 'Recuperação', 'Superaquecimento', 'Estagflação']

# === Bloco 1.3: Helpers ciclo econômico ===
def fetch_fred(series_id, start, end):
    url = (
        'https://api.stlouisfed.org/fred/series/observations'
        f'?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json'
        f'&observation_start={start.strftime("%Y-%m-%d")}'
        f'&observation_end={end.strftime("%Y-%m-%d")}'
    )
    resp = requests.get(url); resp.raise_for_status()
    obs = resp.json().get('observations', [])
    dates = pd.to_datetime([o['date'] for o in obs])
    vals = [float(o['value']) if o['value'] not in ('.','') else np.nan for o in obs]
    return pd.Series(vals, index=dates)


def fetch_indicators():
    data = {}
    gdp = fetch_fred(FRED_SERIES['gdp_real'], START, END).dropna()
    pot = fetch_fred(FRED_SERIES['gdp_pot'], START, END).dropna()
    d = gdp.index.intersection(pot.index)[-1]
    data['gdp_gap'] = (gdp.loc[d]/pot.loc[d] - 1) * 100
    cap = fetch_fred(FRED_SERIES['cap_util'], START, END).dropna()
    data['cap_util_current'] = cap.iloc[-1]
    data['cap_util_trend'] = cap.rolling(12).mean().iloc[-1]
    cpi = fetch_fred(FRED_SERIES['cpi'], START, END).dropna()
    data['inflation_yoy'] = (cpi.iloc[-1]/cpi.shift(12).iloc[-1] - 1) * 100
    r10 = fetch_fred(FRED_SERIES['rate_10y'], START, END).dropna()
    r2 = fetch_fred(FRED_SERIES['rate_2y'], START, END).dropna()
    idx = r10.index.intersection(r2.index)[-1]
    data['yield_curve'] = r10.loc[idx] - r2.loc[idx]
    dxy = yf.download('DX-Y.NYB', start=START, end=END, progress=False)['Close'].dropna()
    data['dxy_current'] = dxy.iloc[-1]
    return {k: float(v) for k,v in data.items()}


def determine_phase(ind):
    gap, inf = ind['gdp_gap'], ind['inflation_yoy']
    if gap < 0 and inf < 2: return 'Reflação'
    if gap >= 0 and inf < 3: return 'Recuperação'
    if gap >= 0 and inf >= 3: return 'Superaquecimento'
    return 'Estagflação'


def display_investment_clock():
    ind = fetch_indicators()
    phase = determine_phase(ind)
    print('🎯 Fase atual:', phase)
    print('🌐 Setores:', CYCLE_MAPPING[phase]['sectors'])
    return ind, {'phase': phase, **CYCLE_MAPPING[phase]}

# === Bloco 2: Carregamento e filtragem de scores ===
SCORES_FOLDER = 'ativos'
SCORE_THRESHOLD = 50.0

def load_and_filter_scores(folder, threshold=SCORE_THRESHOLD):
    files = glob.glob(os.path.join(folder,'*.csv'))
    regs = []
    for fn in files:
        setor = os.path.splitext(os.path.basename(fn))[0]
        df = pd.read_csv(fn, header=None, skiprows=1, usecols=[0,1], names=['Ticker','Score'], decimal=',')
        df['Score'] = pd.to_numeric(df['Score'], errors='coerce')
        df['Setor'] = setor
        regs += [{'Ticker':r.Ticker,'Score':r.Score,'Setor':setor} for r in df.itertuples() if pd.notnull(r.Score)]
    df = pd.DataFrame(regs)
    return df[df['Score']>threshold].sort_values('Score',ascending=False).reset_index(drop=True)

# === Bloco 3: Classificação de tipo de ativo ===
FMP_PROFILE_URL = 'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api_key}'
CACHE = {}

def fetch_asset_profile(ticker):
    t = ticker.upper()
    if t in CACHE: return CACHE[t]
    try:
        resp = requests.get(FMP_PROFILE_URL.format(ticker=t,api_key=FMP_API_KEY)); data = resp.json()
        prof = data[0] if isinstance(data,list) and data else {}
    except:
        prof = {}
    CACHE[t]=prof; return prof


def classify_type(profile):
    m = profile.get('mktCap',0); ind=(profile.get('industry')or'').lower(); nm=(profile.get('companyName')or'').lower()
    if 'etf' in ind or 'etf' in nm: return 'ETF'
    if 'real estate' in ind or 'reit' in ind: return 'REIT'
    if isinstance(m,(int,float)):
        if m>50e9: return 'Blue Chip'
        if m>2e9: return 'Mid Cap'
        return 'Small Cap'
    return 'Unknown'

# === Bloco 4: Geração de ranking para fase atual e próxima fase ===
def generate_report():
    ind, current = display_investment_clock()
    current_phase = current['phase']
    next_idx = (CYCLE_ORDER.index(current_phase)+1) % len(CYCLE_ORDER)
    next_phase = CYCLE_ORDER[next_idx]
    print(f'➡ Próximo ciclo: {next_phase}')

    # Para cada fase (atual e próximo)
    reports = {}
    for phase_name in [current_phase, next_phase]:
        sectors = CYCLE_MAPPING[phase_name]['sectors']
        df = load_and_filter_scores(SCORES_FOLDER)
        df = df[df['Setor'].isin(sectors)].reset_index(drop=True)
        df['Tipo'] = df['Ticker'].apply(lambda t: classify_type(fetch_asset_profile(t)))
        stocks = df[~df['Tipo'].isin(['ETF','REIT'])].head(14).copy(); stocks['Rank']=range(1,len(stocks)+1)
        reits = df[df['Tipo']=='REIT'].head(14).copy(); reits['Rank']=range(1,len(reits)+1)
        print(f"\n=== Top 14 Ações ({phase_name}) ===")
        print(stocks[['Rank','Ticker','Score','Setor']].to_string(index=False) if not stocks.empty else 'Nenhuma ação')
        print(f"\n=== Top 14 REITs ({phase_name}) ===")
        print(reits[['Rank','Ticker','Score','Setor']].to_string(index=False) if not reits.empty else 'Nenhum REIT')
        reports[phase_name] = (stocks, reits)

    return reports

# === Bloco 5: Agendamento semanal ===
def schedule_weekly_report(day='monday',time_str='09:00'):
    getattr(schedule.every(),day).at(time_str).do(generate_report)
    print(f'Relatório agendado: toda {day} às {time_str}')

if __name__=='__main__':
    generate_report()


python-dotenv could not parse statement starting at line 8


🎯 Fase atual: Recuperação
🌐 Setores: ['Telecoms', 'Information Technology', 'Basic Materials', 'Consumer Discretionary']
➡ Próximo ciclo: Superaquecimento

=== Top 14 Ações (Recuperação) ===
 Rank Ticker  Score                  Setor
    1    CRM  73.88 Information Technology
    2   RACE  70.87 Consumer Discretionary
    3    YUM  67.83 Consumer Discretionary
    4      V  65.73 Information Technology
    5    TJX  65.73 Consumer Discretionary
    6    MCD  64.00 Consumer Discretionary
    7   ORLY  63.65 Consumer Discretionary
    8   TMUS  62.88 Information Technology
    9    ADI  62.53 Information Technology
   10     IP  61.39        Basic Materials
   11    CMG  61.38 Consumer Discretionary
   12    LOW  61.00 Consumer Discretionary
   13    IBM  60.89 Information Technology
   14    APH  60.14 Information Technology

=== Top 14 REITs (Recuperação) ===
Nenhum REIT

=== Top 14 Ações (Superaquecimento) ===
 Rank Ticker  Score                  Setor
    1    CRM  73.88 Information 